In [46]:
!ls ../input/flowers-recognition/flowers

daisy  dandelion  flowers  rose  sunflower  tulip


In [47]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

In [48]:
device= torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [49]:
transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                                transforms.RandomRotation(0.2),
                                transforms.ToTensor(),
                                transforms.Resize((80,80))
                               ])

In [50]:
data= torchvision.datasets.ImageFolder(root="../input/flowers-recognition/flowers", transform=transform)

In [51]:
len(data)

8646

In [52]:
train_set, val_set = torch.utils.data.random_split(data, [6917, 1729])

batch_size = 40
train_loader = torch.utils.data.DataLoader(dataset=train_set,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=val_set,
                                          batch_size=batch_size, 
                                          shuffle=True)


In [53]:
# Hyper parameters
num_epochs = 15
num_classes = 5
learning_rate = 0.001


In [54]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer4 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer5 = nn.Sequential(
            nn.Conv2d(512, 1024, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(1024),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc1 = nn.Linear(2*2*1024, 256)
        self.fc2 = nn.Linear(256, 512)
        self.fc3 = nn.Linear(512, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = F.dropout(out, training=self.training)
        out = self.fc2(out)
        out = F.dropout(out, training=self.training)
        out = self.fc3(out)
        return F.log_softmax(out,dim=1)

In [55]:
model = ConvNet().to(device)


In [56]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [57]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    training_accuracy=0
    for i, (images, labels) in enumerate(train_loader):
        model.train()
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Checking accuracy
        preds = outputs.data.max(dim = 1, keepdim = True)[1]
        training_accuracy += preds.eq(labels.data.view_as(preds)).cpu().sum()
        
        
    training_accuracy = training_accuracy/len(train_loader.dataset) * 100
        
    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Training Accuracy: {}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item(), training_accuracy))


Epoch [1/15], Step [173/173], Loss: 1.3834, Training Accuracy: 42.70637512207031
Epoch [2/15], Step [173/173], Loss: 1.1471, Training Accuracy: 46.711002349853516
Epoch [3/15], Step [173/173], Loss: 1.3390, Training Accuracy: 46.985687255859375
Epoch [4/15], Step [173/173], Loss: 1.0681, Training Accuracy: 46.87002944946289
Epoch [5/15], Step [173/173], Loss: 1.0346, Training Accuracy: 48.171173095703125
Epoch [6/15], Step [173/173], Loss: 1.1294, Training Accuracy: 48.14225769042969
Epoch [7/15], Step [173/173], Loss: 1.1412, Training Accuracy: 47.202545166015625
Epoch [8/15], Step [173/173], Loss: 1.3548, Training Accuracy: 48.229000091552734
Epoch [9/15], Step [173/173], Loss: 1.0164, Training Accuracy: 48.95185852050781
Epoch [10/15], Step [173/173], Loss: 1.0389, Training Accuracy: 48.156715393066406
Epoch [11/15], Step [173/173], Loss: 1.0490, Training Accuracy: 48.214542388916016
Epoch [12/15], Step [173/173], Loss: 1.0581, Training Accuracy: 48.24345779418945
Epoch [13/15], Ste

In [58]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    testing_accuracy=0
    for i, (images, labels) in enumerate(test_loader):
        model.eval()
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Checking accuracy
        preds = outputs.data.max(dim = 1, keepdim = True)[1]
        testing_accuracy += preds.eq(labels.data.view_as(preds)).cpu().sum()
        
        
    testing_accuracy = testing_accuracy/len(test_loader.dataset) * 100
        
    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Testing Accuracy: {}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item(), testing_accuracy))


Epoch [1/15], Step [44/173], Loss: 1.2035, Testing Accuracy: 49.045692443847656
Epoch [2/15], Step [44/173], Loss: 1.1307, Testing Accuracy: 49.624061584472656
Epoch [3/15], Step [44/173], Loss: 0.9670, Testing Accuracy: 49.97108459472656
Epoch [4/15], Step [44/173], Loss: 1.1071, Testing Accuracy: 50.665122985839844
Epoch [5/15], Step [44/173], Loss: 0.8478, Testing Accuracy: 51.87969970703125
Epoch [6/15], Step [44/173], Loss: 1.5495, Testing Accuracy: 50.8964729309082
Epoch [7/15], Step [44/173], Loss: 0.9214, Testing Accuracy: 51.53268051147461
Epoch [8/15], Step [44/173], Loss: 0.9136, Testing Accuracy: 52.2845573425293
Epoch [9/15], Step [44/173], Loss: 1.0601, Testing Accuracy: 53.267784118652344
Epoch [10/15], Step [44/173], Loss: 0.7759, Testing Accuracy: 57.72122573852539
Epoch [11/15], Step [44/173], Loss: 0.6092, Testing Accuracy: 56.96934509277344
Epoch [12/15], Step [44/173], Loss: 1.0219, Testing Accuracy: 59.340660095214844
Epoch [13/15], Step [44/173], Loss: 0.9078, Te

In [59]:
# Train the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Train Accuracy of the model on the 8000 test images with Relu is: {} %'.format(100 * correct / total))


Train Accuracy of the model on the 8000 test images with Relu is: 42.489518577417954 %


In [60]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 8000 test images with Relu is: {} %'.format(100 * correct / total))


Test Accuracy of the model on the 8000 test images with Relu is: 63.67842683632157 %


# Applying Transfer Learning with vgg

In [61]:
transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                                transforms.RandomRotation(0.2),
                                transforms.ToTensor(),
                                transforms.Resize((80,80))
                               ])

In [62]:
data= torchvision.datasets.ImageFolder(root="../input/flowers-recognition/flowers", transform=transform)

In [63]:
train_set, val_set = torch.utils.data.random_split(data, [6917, 1729])

batch_size = 40
train_loader = torch.utils.data.DataLoader(dataset=train_set,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=val_set,
                                          batch_size=batch_size, 
                                          shuffle=True)


In [64]:
vgg = torchvision.models.vgg19(pretrained=True)


Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

In [65]:
vgg

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [66]:
vgg.classifier[6].out_features = 5
for param in vgg.features.parameters(): 
    param.requires_grad = False

vgg = vgg.cuda()

In [67]:
# Hyper parameters
num_epochs = 15
num_classes = 5
learning_rate = 0.001


In [68]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vgg.parameters(), lr=learning_rate)

In [69]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    training_accuracy=0
    for i, (images, labels) in enumerate(train_loader):
        vgg.train()
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = vgg(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Checking accuracy
        preds = outputs.data.max(dim = 1, keepdim = True)[1]
        training_accuracy += preds.eq(labels.data.view_as(preds)).cpu().sum()
        
        
    training_accuracy = training_accuracy/len(train_loader.dataset) * 100
        
    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Training Accuracy: {}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item(), training_accuracy))


Epoch [1/15], Step [173/173], Loss: 1.1704, Training Accuracy: 43.34248733520508
Epoch [2/15], Step [173/173], Loss: 1.6498, Training Accuracy: 45.42431640625
Epoch [3/15], Step [173/173], Loss: 1.7145, Training Accuracy: 44.10871887207031
Epoch [4/15], Step [173/173], Loss: 1.5155, Training Accuracy: 45.323116302490234
Epoch [5/15], Step [173/173], Loss: 1.0113, Training Accuracy: 46.45077133178711
Epoch [6/15], Step [173/173], Loss: 1.0431, Training Accuracy: 45.88694763183594
Epoch [7/15], Step [173/173], Loss: 1.1087, Training Accuracy: 46.85557174682617
Epoch [8/15], Step [173/173], Loss: 1.4245, Training Accuracy: 47.27482986450195
Epoch [9/15], Step [173/173], Loss: 1.3515, Training Accuracy: 46.566429138183594
Epoch [10/15], Step [173/173], Loss: 1.2281, Training Accuracy: 47.65071487426758
Epoch [11/15], Step [173/173], Loss: 1.0031, Training Accuracy: 47.70854568481445
Epoch [12/15], Step [173/173], Loss: 0.9461, Training Accuracy: 46.783287048339844
Epoch [13/15], Step [173/

In [70]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    testing_accuracy=0
    for i, (images, labels) in enumerate(test_loader):
        vgg.eval()
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = vgg(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Checking accuracy
        preds = outputs.data.max(dim = 1, keepdim = True)[1]
        testing_accuracy += preds.eq(labels.data.view_as(preds)).cpu().sum()
        
        
    testing_accuracy = testing_accuracy/len(test_loader.dataset) * 100
        
    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Testing Accuracy: {}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item(), testing_accuracy))


Epoch [1/15], Step [44/173], Loss: 1.6417, Testing Accuracy: 49.27703857421875
Epoch [2/15], Step [44/173], Loss: 1.5318, Testing Accuracy: 51.937538146972656
Epoch [3/15], Step [44/173], Loss: 1.1291, Testing Accuracy: 53.7304801940918
Epoch [4/15], Step [44/173], Loss: 0.6046, Testing Accuracy: 58.82012939453125
Epoch [5/15], Step [44/173], Loss: 0.7793, Testing Accuracy: 64.43030548095703
Epoch [6/15], Step [44/173], Loss: 0.5819, Testing Accuracy: 67.72701263427734
Epoch [7/15], Step [44/173], Loss: 0.6094, Testing Accuracy: 71.02371215820312
Epoch [8/15], Step [44/173], Loss: 0.4536, Testing Accuracy: 76.11335754394531
Epoch [9/15], Step [44/173], Loss: 0.6674, Testing Accuracy: 77.55928039550781
Epoch [10/15], Step [44/173], Loss: 1.0779, Testing Accuracy: 76.80740356445312
Epoch [11/15], Step [44/173], Loss: 0.5100, Testing Accuracy: 81.08733367919922
Epoch [12/15], Step [44/173], Loss: 0.6687, Testing Accuracy: 82.24407196044922
Epoch [13/15], Step [44/173], Loss: 0.2389, Testi

In [71]:
# Train the model
vgg.train()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = vgg(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Train Accuracy of the model on the 8000 test images with Relu is: {} %'.format(100 * correct / total))


Train Accuracy of the model on the 8000 test images with Relu is: 40.29203411883765 %


In [72]:
# Test the model
vgg.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = vgg(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 8000 test images with Relu is: {} %'.format(100 * correct / total))


Test Accuracy of the model on the 8000 test images with Relu is: 88.37478311162522 %
